# Tworzenie asystenta wyszukiwania plików (GPT-4o), który podsumowuje i wyjaśnia artykuły dotyczące AGI na portalu arxiv.

## Kroki wstępne

- Upewnij się, że masz konto deweloperskie OpenAI z dostępnymi środkami finansowymi.
- Zdefiniuj zmienną środowiskową o nazwie `OPENAI_API_KEY` zawierającą klucz API.

## Zadanie 0: Setup

Najpierw musimy upewnić się, że korzystamy z najnowszej wersji pakietu OpenAI API.

In [1]:
# Run this to install the latest version of the OpenAI package
!pip install openai==1.33.0

Potrzebujemy paczek `os`, `openai`, `pandas`.

### Instrukcje

- Zaimportuj paczki `os`, `openai` bez aliasu.
- Zaimportuj paczkę `pandas` z wykorzystaniem zwyczajowego aliasu dla tej paczki.

In [2]:
# Import the os package


# Import the openai package


# Import the pandas package with an alias


Musimy zdefiniować klienta OpenAI.

### Instrukcje

- Zdefiniuj klienta OpenAI. Przypisz do: `client`.

In [3]:
# Define an OpenAI client. Assign to client.


## Zadanie 1: Upload dokumentów

Aby GPT wiedział o najnowszych badaniach nad AGI, dostarczymy mu kilka artykułów z arxiv. W katalogu `papers` znajduje się 10 najnowszych artykułów na temat AGI.

Poniższa tabela pokazuje nazwy plików i tytuły artykułów.

In [4]:
# Run this
papers = pd.DataFrame({
    "filename": [
        "2405.10313v1.pdf",
        "2401.03428v1.pdf",
        "2401.09395v2.pdf",
        "2401.13142v3.pdf",
        "2403.02164v2.pdf",
        "2403.12107v1.pdf",
        "2404.10731v1.pdf",
        "2312.11562v5.pdf",
        "2311.02462v2.pdf",
        "2310.15274v1.pdf"
    ],
    "title": [
        "How Far Are We From AGI?",
        "EXPLORING LARGE LANGUAGE MODEL BASED INTELLIGENT AGENTS: DEFINITIONS, METHODS, AND PROSPECTS",
        "CAUGHT IN THE QUICKSAND OF REASONING, FAR FROM AGI SUMMIT: Evaluating LLMs’ Mathematical and Coding Competency through Ontology-guided Interventions",
        "Unsocial Intelligence: an Investigation of the Assumptions of AGI Discourse",
        "Cognition is All You Need The Next Layer of AI Above Large Language Models",
        "Scenarios for the Transition to AGI",
        "What is Meant by AGI? On the Definition of Artificial General Intelligence",
        "A Survey of Reasoning with Foundation Models",
        "Levels of AGI: Operationalizing Progress on the Path to AGI",
        "Systematic AI Approach for AGI: Addressing Alignment, Energy, and AGI Grand Challenges"
    ]
})
papers["filename"] = "papers/" + papers["filename"]
papers

Aby przesłać plik, należy użyć funkcji `open()`, a rezultat przekazać do metody `.files.create()`. Zwraca ona szczegóły przesłanego pliku wraz z potrzebnym nam identyfikatorem.

### Instrukcje

- Uruchom ten kod, aby zdefiniować funkcję przesyłania pliku do asystenta.

In [5]:
# Run this
def upload_file_for_assistant(file_path): 
    uploaded_file = client.files.create(
        file=open(file_path, "rb"),
        purpose='assistants'
    )
    return uploaded_file.id

Teraz stosujemy funkcję `upload_file_for_assistant()` do każdej nazwy pliku w zbiorze danych dokumentów, aby je przesłać.

### Instrukcje

- W `papers` wybierz kolumnę `filename`, następnie zastosuj `upload_file_for_assistant()`, a następnie przekonwertuj wynik na listę. Przypisz do `uploaded_file_ids`.

In [6]:
# In papers, select the filename column, 
# then apply upload_file_for_assistant(),
# then convert the result to a list. 
# Assign to uploaded_file_ids.


# See the result
uploaded_file_ids

### Sprawdź, czy to zadziałało

Wyświetl pliki na swoim koncie pod adresem https://platform.openai.com/storage/files

## Zadanie 2: Dodawanie plików do Vector Store

Aby uzyskać dostęp do dokumentów i uzyskać sensowne wyniki, należy je podzielić na małe fragmenty (chunks) i dodać do wektorowej bazy danych.

Interfejs API asystentów pozwala uniknąć martwienia się o etap dzielenia, wystarczy określić identyfikatory plików, które mają zostać dodane do wektorowej bazy danych.

#### Uwagi

- Opłata za posiadanie wektorowej bazy danych będzie naliczana codziennie. Domyślnie zostanie ona automatycznie usunięta po 7 dniach nieużywania.

### Instrukcje

- Utwórz Vector store, kojarząc identyfikatory przesłanych plików i nadając nazwę. (Sugerowana nazwa: `arxiv_agi_papers`.) Przypisz do `vstore`.

<details>
  <summary>Podpowiedzi</summary>
  <p>
        
```py
vstore = client.beta.vector_stores.create(
    file_ids = file_ids,
    name = "vector store name"
)
```
        
  </p>
</details>   

In [7]:
# Create a vector store, associating the uploaded file IDs and naming it.


# See the results
vstore

### Sprawdź, czy to zadziałało

Wyświetl Vector stores na swoim koncie pod adresem https://platform.openai.com/storage/vector_stores.

## Zadanie 3: Tworzenie asystenta

Asystent potrzebuje podpowiedzi opisującej, jak powinien się zachowywać. Składa się ona z kilku akapitów tekstu, które dostarczają GPT informacji o tym, jaka jest jego rola, o czym powinien mówić i jak formułować odpowiedzi.

#### Pro tip

Podobnie jak każdy inny tekst, podpowiedź asystenta może zostać wygenerowana za pomocą ChatGPT (lub dowolnego LLM). Poniższa podpowiedź została przygotowana przez ChatGPT i została poddana jedynie niewielkiej edycji.

Oto podpowiedź ChatGPT, której użyłem do stworzenia podpowiedzi asystenta.

> I'm going to make a GPT assistant that explains the contents of journal articles about artificial general intelligence. The assistant, named 'Aggie', must be able to read arxiv papers in PDF form, and and explain the contents of those papers to an audience of data scientists. Please suggest a good instruction prompt for the AI assistant.

### Instrukcje

- Przeczytaj tekst opisu asystenta, aby zrozumieć jego działanie.
- Uruchom kod, aby zdefiniować prompt asystenta.

In [8]:
# Run this
assistant_prompt = """
You are Aggie, a knowledgeable and articulate AI assistant specializing in artificial general intelligence (AGI). Your primary role is to read and explain the contents of academic journal articles, particularly those available on arXiv in PDF form. Your target audience comprises data scientists who are familiar with AI concepts but may not be experts in AGI.

When explaining the contents of the papers, follow these guidelines:

Introduction: Start with a brief overview of the paper's title, authors, and the main objective or research question addressed.

Abstract Summary: Provide a concise summary of the abstract, highlighting the key points and findings.

Key Sections and Findings: Break down the paper into its main sections (e.g., Introduction, Methods, Results, Discussion). For each section, provide a summary that includes:

The main points and arguments presented.
Any important methods or techniques used.
Key results and findings.
The significance and implications of these findings.
Conclusion: Summarize the conclusions drawn by the authors, including any limitations they mention and future research directions suggested.

Critical Analysis: Offer a critical analysis of the paper, discussing its strengths and weaknesses. Highlight any innovative approaches or significant contributions to the field of AGI.

Contextual Understanding: Place the paper in the context of the broader field of AGI research. Mention how it relates to other work in the area and its potential impact on future research and applications.

Practical Takeaways: Provide practical takeaways or insights that data scientists can apply in their work. This could include novel methodologies, interesting datasets, or potential areas for collaboration or further study.

Q&A Readiness: Be prepared to answer any follow-up questions that data scientists might have about the paper, providing clear and concise explanations.

Ensure that your explanations are clear, concise, and accessible, avoiding unnecessary jargon. Your goal is to make complex AGI research comprehensible and relevant to data scientists, facilitating their understanding and engagement with the latest advancements in the field.
"""

Teraz można utworzyć asystenta. Wystarczy nadać mu nazwę, prompt, model (w tym przypadku GPT-4o) i określić, z jakich narzędzi i zasobów może korzystać.

### Instrukcje

- Zdefiniuj asystenta. Przypisz do `aggie`.
    - Nazwij go "Aggie" (lub inną zapadającą w pamięci nazwą).
    - Nadaj mu `assistant_prompt`.
    - Ustaw model do użycia, `gpt-4o`.
    - Daj mu dostęp do narzędzia wyszukiwania plików.
    - Daj mu dostęp do zasobu narzędzia vector store.

<details>
  <summary>Podpowiedzi</summary>
  <p>
       
```py
assistant = client.beta.assistants.create(
	name = "assistant name",
	instructions = prompt,
	model="gpt-4o",
	tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vstore.id]}}
)
```
        
  </p>
</details>   

In [9]:
# Define the assistant. Assign to aggie.

    
# See the result
aggie

### Sprawdź, czy to zadziałało

Wyświetl stworzonych asystentów na swoim koncie pod adresem https://platform.openai.com/playground/assistants.

## Zadanie 4: Utwórz wątek konwersacji

Mamy już asystenta, możemy zatem zaczac rozmowę. Pierwszym krokiem jest utworzenie obiektu wątku zawierającego wiadomości.

### Instrukcje

- Utwórz obiekt wątku. Przypisz do `conversation`.

<details>
  <summary>Podpowiedzi</summary>
  <p>

Aby utworzyć obiekt konwersacji, wywołaj `client.beta.threads.create()`.
        
  </p>
</details>  

In [10]:
# Create a thread object. Assign to conversation.


# See the result
conversation

Następnie możesz dodać wiadomość do wątku konwersacji, aby zadać pytanie.

### Instrukcje

- Dodaj wiadomość użytkownika do konwersacji. Przypisz do `msg_what_is_agi`.
    - Nadaj jej identyfikator wątku (wiadomość użytkownika).
    - Zadaj pytanie "What are the most common definitions of AGI?".

<details>
  <summary>Podpowiedzi</summary>
  <p>
        
```py
msg = client.beta.threads.messages.create(
    thread_id=conversation.id,
    role="user",
    content="your question"
)
```
        
  </p>
</details>   

In [11]:
# Add a user message to the conversation. Assign to msg_what_is_agi.


# See the result
msg_what_is_agi

## Zadanie 6: Uruchamianie asystenta

Uruchomienie asystenta wymaga obsługi zdarzenia (handler), aby wyświetlić odpowiedzi. Chociaż poniższy kod jest dość skomplikowany, nigdy nie trzeba go zmieniać. Ten kod pochodzi dosłownie z [dokumentacji asystentów OpenAI] (https://platform.openai.com/docs/assistants/overview).

### Instrukcje

- Uruchom kod, aby zdefiniować obsługę zdarzeń.

In [12]:
# Run this
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


Wreszcie jesteśmy gotowi do uruchomienia asystenta, aby uzyskać odpowiedź na nasze pytanie. Kod jest taki sam za każdym razem, więc możemy zamienić go w funkcję.

Strumieniowe (stream) odpowiedzi oznaczają, że tekst jest wyświetlany po kilka słów na raz, zamiast czekać na wygenerowanie i wyświetlenie całego tekstu naraz.

### Instrukcje

- Uruchom kod, aby zdefiniować funkcję.

In [13]:
# Run this
def run_aggie():
    with client.beta.threads.runs.stream(
        thread_id=conversation.id,
        assistant_id=aggie.id,
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

### Instrukcje

- Uruchom asystenta.

In [14]:
# Run the assistant


## Zadanie 6: Dodaj kolejną wiadomość i uruchom ją ponownie

Skoro już zadaliśmy sobie trud stworzenia asystenta, równie dobrze możemy zadać więcej pytań.

### Instrukcje

- Utwórz kolejną wiadomość użytkownika, dodając ją do konwersacji. Tym razem zapytaj "How close are we to developing AGI?. Przypisz do `msg_how_close_is_agi`.

In [15]:
# Create another user message, adding it to the conversation. Assign to msg_how_close_is_agi.


# See the result
msg_how_close_is_agi

### Instrukcje

- Uruchom ponownie asystenta.

In [16]:
# Run the assistant
